# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*


#### Описание данных : <br>
Данные геологоразведки трёх регионов находятся в трех файлах:
   - geo_data_0 - первый регион;
   - geo_data_1 - второй регион;
   - geo_data_2 - третий регион;

Каждый файл состоит из следующих столбцов:<br>

- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).


## Загрузка и подготовка данных

In [ ]:
pip install phik

In [ ]:
# импортируем необходимые для работы библиотеки
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
#import phik
#from phik.report import plot_correlation_matrix
#from phik import report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [ ]:
# Основные характеристики для расчетов:
BUDGET=10E9         # общий бюджет на разработку
BARREL_PRICE=450000 # стоимость одной тысячи баррелей
TOP_HOLES=200       # количество скважин c максимальным запасом по предсказанию
HOLES=500           # количество скважин для разработки
BOOTSTRAP_SIZE=1000  

In [ ]:
try:
    df_1 = pd.read_csv(r'D:\project ML в бизнесе\geo_data_0.csv')
    df_2 = pd.read_csv(r'D:\project ML в бизнесе\geo_data_1.csv')
    df_3 = pd.read_csv(r'D:\project ML в бизнесе\geo_data_2.csv')
except:
    df_1 = pd.read_csv('/datasets/geo_data_0.csv')
    df_2 = pd.read_csv('/datasets/geo_data_1.csv')
    df_3 = pd.read_csv('/datasets/geo_data_2.csv')

#### Изучение и подготовка данных для первого региона.

In [ ]:
display(df_1.head()), display(df_1.info())

Посмотрим основные статистики нашей выборки:

In [ ]:
df_1.describe()

Проведем проверку на наличие явных дубликатов: 

In [ ]:
df_1.head()

In [ ]:
df_1['new'] = df_1['f0']* 100

In [ ]:
df_1.head()

In [ ]:
df_1.duplicated().sum()

In [ ]:
df_1[['f0','f1','f2','product']].corr()

In [ ]:
df_1[['f0','f1','f2','product']].phik_matrix()

In [ ]:
df_1['f0'].plot(kind='hist',grid=True,title='Признак "f0" первого региона');

In [ ]:
df_1['f1'].plot(kind='hist',grid=True,title='Признак "f1" первого региона');

In [ ]:
df_1['f2'].plot(kind='hist',grid=True,title='Признак "f2" первого региона');

In [ ]:
df_1['product'].plot(kind='hist',grid=True,title='Целевой признак "product" первого региона');
plt.xlabel('тыс.бар.');

### Подготовка данных для второго региона.

In [ ]:
display(df_2.head()), display(df_2.info())

Посмотрим основные статистики нашей выборки:

In [ ]:
df_2.describe()

Проведем проверку на наличие явных дубликатов: 

In [ ]:
df_2.duplicated().sum()

In [ ]:
df_2[['f0','f1','f2','product']].corr()

In [ ]:
df_2[['f0','f1','f2','product']].phik_matrix()

In [ ]:
df_2['f0'].plot(kind='hist',grid=True,title='Признак "f0" второго региона');

In [ ]:
df_2['f1'].plot(kind='hist',grid=True,title='Признак "f1" второго региона');

In [ ]:
df_2['f2'].plot(kind='hist',grid=True,title='Признак "f2" второго региона');

In [ ]:
df_2['product'].plot(kind='hist',grid=True,title='Целевой признак "product" второго региона');
plt.xlabel('тыс.бар.');

### Подготовка данных для третьего региона.

In [ ]:
display(df_3.head()), display(df_3.info())

Посмотрим основные статистики нашей выборки:

In [ ]:
df_3.describe()

Проведем проверку на наличие явных дубликатов: 

In [ ]:
df_3.duplicated().sum()

In [ ]:
df_3[['f0','f1','f2','product']].corr()

In [ ]:
df_3[['f0','f1','f2','product']].phik_matrix()

In [ ]:
df_3['f0'].plot(kind = 'hist',grid = True,title='Признак "f0" третьего региона');

In [ ]:
df_3['f1'].plot(kind='hist',grid=True,title='Признак "f1" третьего региона');

In [ ]:
df_3['f2'].plot(kind='hist',grid=True,title='Признак "f2" третьего региона');

In [ ]:
df_3['product'].plot(kind='hist',grid=True,title='Целевой признак "product" третьего региона');
plt.xlabel('тыс.бар.');

## Обучение и проверка модели

In [ ]:
def modeling(df):
    
    # выделяем целевой признак и признаки для прогноза
    X=df.drop(['id','product'], axis=1)
    y=df['product']
    
    # делим выборку на тренировочную и валидационную выборки в соотношении 75:25
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)
    
    # обучаем модель линейной регрессии
    model = LinearRegression()
    model.fit(X_train, y_train)
    pred = model.predict(X_valid)
    mse = mean_squared_error(pred, y_valid)
    rmse = np.sqrt(mse)
    mean_values=pred.mean()
    return y_valid, pred, mse, rmse, mean_values

#### Первый регион.

In [ ]:
y_valid_1, pred_1, mse_1, rmse_1, mean_values_1 = modeling(df_1)

In [ ]:
print('RMSE_1:', rmse_1)
print('Средний запас предсказанного сырья:', mean_values_1,'тыс.бар.')

#### Второй регион.

In [ ]:
y_valid_2, pred_2, mse_2, rmse_2, mean_values_2 = modeling(df_2)

In [ ]:
print('RMSE_2:', rmse_2)
print('Средний запас предсказанного сырья:', mean_values_2,'тыс.бар.')

#### Третий регион.

In [ ]:
y_valid_3, pred_3, mse_3, rmse_3, mean_values_3 = modeling(df_3)

In [ ]:
print('RMSE_3:', rmse_3)
print('Средний запас предсказанного сырья:', mean_values_3,'тыс.бар.')

Средний запас сырья по предсказанным данным больше у третьего региона, но среднеквадратическая ошибка здесь так же самая большаяя.

## Подготовка к расчёту прибыли

#### Функция для расчета прибыли.

In [ ]:
min_price=np.round(BUDGET/(BARREL_PRICE*TOP_HOLES)) 
print(min_price,'тыс.бар.')

- Достаточный объём сырья для безубыточной разработки новой скважины= 111 тыс.баррелей

#### Сравним средним запас по предсказаниям модели в каждом регионе  с достаточным объёмом сырья для безубыточной разработки новой скважины.

In [ ]:
print('Достаточный объём:', min_price,'тыс.бар.')
print('Первый регион:', np.round(y_valid_1.mean(),decimals=2),'тыс.бар.')
print('Второй регион:', np.round(y_valid_2.mean(),decimals=2),'тыс.бар.')
print('Третий регион:', np.round(y_valid_3.mean(),decimals=2),'тыс.бар.')

- Во всех регионах средний запас сырья по предсказаниям модели меньше достаточного объёма сырья для безубыточной разработки новой скважины.

## Расчёт прибыли и рисков 

Подготовка типов данных:

In [ ]:
def type_series(y, pred):
    y=y.reset_index(drop=True)
    pred=pd.Series(pred)
    return y, pred

In [ ]:
y_valid_1, pred_1=type_series(y_valid_1, pred_1)

In [ ]:
y_valid_2, pred_2=type_series(y_valid_2, pred_2)

In [ ]:
y_valid_3, pred_3=type_series(y_valid_3, pred_3)

#### Посчитаем прибыль по выбранным скважинам и предсказаниям модели.

In [ ]:
#функция для расчёта прибыли по выбранным скважинам и предсказаниям модели
def profit(y: pd.Series, pred: pd.Series)-> int:
    pred_top = pred.sort_values(ascending = False)
    selected = y[pred_top.index][:TOP_HOLES]
    value = selected.sum()
    profit = value*BARREL_PRICE-BUDGET 
    return profit

Первый регион

In [ ]:
print(np.round(profit(y_valid_1, pred_1)/1000000,decimals=2),',млн.руб.')

Второй регион

In [ ]:
print(np.round(profit(y_valid_2, pred_2)/1000000,decimals=2),',млн.руб.')

Третий регион

In [ ]:
print(np.round(profit(y_valid_3, pred_3)/1000000,decimals=2),',млн.руб.')

#### Применим технику Bootstrap c 1000 выборок, чтобы найти распределение прибыли. Также найдем среднюю прибыль, 95%-ый доверительный интервал и риск убытков.

In [ ]:
def bootstrap(y_valid: pd.Series , pred: pd.Series):
    state = np.random.RandomState(12345)
    values_profit = []
    for i in range(BOOTSTRAP_SIZE):
        y_subsample = y_valid.sample(n = HOLES, replace = True, random_state=state)
        pred_subsumple = pred[y_subsample.index]
        pred_subsumple = pred_subsumple.reset_index(drop = True)
        y_subsample = y_subsample.reset_index(drop = True)
        profit_dist = profit(y_subsample, pred_subsumple)
        values_profit.append(profit_dist/1000000)
    values_profit = pd.Series(values_profit)    
    profit_mean = np.round(values_profit.mean(),decimals=2),
    lower = np.round(values_profit.quantile(0.25),decimals=2),
    higher = np.round(values_profit.quantile(0.975),decimals=2),
    risk = ((values_profit < 0).mean())*100
    return values_profit, profit_mean, lower, higher, risk

#### Первый регион

In [ ]:
values_profit_1, profit_mean_1, lower_1, higher_1, risk_1 = bootstrap(y_valid_1,pred_1)

In [ ]:
values_profit_1.plot(kind='hist',title='Распределение прибыли в первом регионе',grid=True);
plt.xlabel('млн.руб.');

#### Второй регион

In [ ]:
values_profit_2, profit_mean_2, lower_2, higher_2, risk_2 = bootstrap(y_valid_2,pred_2)

In [ ]:
values_profit_1.plot(kind='hist',title='Распределение прибыли во втором регионе',grid=True);
plt.xlabel('млн.руб.');

#### Третий регион

In [ ]:
values_profit_3, profit_mean_3, lower_3, higher_3, risk_3 = bootstrap(y_valid_3,pred_3)

In [ ]:
values_profit_3.plot(kind='hist',title='Распределение прибыли в третьем регионе',grid=True);
plt.xlabel('млн.руб.');

In [ ]:
final=pd.DataFrame(
                  [
                   [profit_mean_1, lower_1, higher_1, risk_1],
                   [profit_mean_2, lower_2, higher_2, risk_2],
                   [profit_mean_3, lower_3, higher_3, risk_3]],
                   index=['Первый регион','Второй регион','Третий регион'],
                   columns=['Средняя прибыль,млн.руб','Нижняя граница доверительно интервала,млн.руб.',
                            'Верхний граница доверительно интервала,млн.руб.','Риск убытков,%'
                                                                                                        ]
                                                                                                        )

In [ ]:
display(final)

### Вывод:<br>
В рамках данного исследования была проведена подготовка данных, обучена модель линейной регрессии и сделан прогноз запасов для скважин из трёх регионов:<br> 
* Регион 1. Средний запас сырья: 92.33 тыс. баррелей*   
* Регион 2. Средний запас сырья: 68.73 тыс. баррелей*  
* Регион 3. Средний запас сырья: 95.15 тыс. баррелей*  
Достаточный объём: 1111.0

Достаточный объём сырья для безубыточной разработки - новой скважины равен 111 тыс. барреля.

В каждом из трёх регионов средний запас сырья скважины меньше полученного значения достаточного объёма сырья.

#### Прибыль по выбранным скважинам и предсказаниям модели. <br>

 - Регион 1- 3359,14 ,млн.руб.
 - Регион 2- 2415 ,млн.руб. 
 - Реuион 3- 2598,57 ,млн.руб.

#### Применили технику Bootstrap с 1000 выборок, для нахождения распределение прибыли. <br>

In [ ]:
display(final)

На основании полученных данных можно сделать вывод, что `регион 2` является наиболее привлекательным для разработки. Самая высокая прибыль лучших месторождений и наименьший риск получения убытков..  

Данный регион рекомендован к освоению.  